In [1]:
import gym

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random

import frame_player

class A2CNet(nn.Module):
    def __init__(self, in_size, out_size, hidden = 128):
        super(A2CNet, self).__init__()
        self.policy = nn.Sequential(
            nn.Linear(in_size, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_size)
        )
        self.value = nn.Sequential(
            nn.Linear(in_size, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )
    
    def forward(self, x):
        return self.policy(x), self.value(x)

EPOCH = 5000
BATCH = 4
GAME = "MountainCar-v0"

LR = 0.0007
GAMMA = 0.99
ENT_SCALE = 10

VIDEO = 50

CLIP = 5
    
env = gym.make(GAME)
env._max_episode_steps = 1000

net = A2CNet(env.observation_space.shape[0], env.action_space.n)
opt = optim.Adam(net.parameters(), lr=LR)

In [ ]:
if __name__ == "__main__":
    batch_cnt = 0
    batch_obs = []
    batch_act = []
    batch_qval = []
    for epoch in range(EPOCH):
        count = 0
        obs = env.reset()
        if epoch%VIDEO == 0:
            env.render()
        cur_rew = []
        while True:
            with torch.no_grad():
                logit, _ = net(torch.FloatTensor(obs))
                act = np.random.choice(env.action_space.n, p=F.softmax(logit).numpy())
                
            next_obs, rew, done, _ = env.step(act)
            if epoch%VIDEO == 0:
                env.render()
            count+=1
            
            batch_obs.append(obs)
            batch_act.append(act)
            cur_rew.append(rew)
            
            obs = next_obs
            if done:
                qval = []
                r_sum = 0
                for r in reversed(cur_rew):
                    r_sum*= GAMMA
                    r_sum+= r
                    qval.append(r_sum)
                batch_qval.extend(list(reversed(qval)))
                batch_cnt+=1
                break
        print("epoch : %d count : %d rew : %d"%(epoch, count, rew))
        if batch_cnt == BATCH:
            obss = torch.FloatTensor(batch_obs)
            acts = torch.LongTensor(batch_act).unsqueeze(1)
            qvals = torch.FloatTensor(batch_qval).unsqueeze(1)
            
            opt.zero_grad()
            
            # use network to get logit, value
            logit, value = net(obss)
            
            # value -> need to be similar to qvals.
            value_loss = F.mse_loss(value, qvals)
            
            # policy gradient. using advatange score to reduce variance.
            log_prob = F.log_softmax(logit, dim=1).gather(1, acts)
            adv = qvals - value.detach()
            policy_value = adv * log_prob
            policy_loss = -policy_value.mean()
            
            # using entropy to make agent explore more.
            prob = F.softmax(logit, dim=1).gather(1, acts)
            entropy = - prob * log_prob 
            entropy_loss = - ENT_SCALE * entropy.mean()
            
            loss = value_loss + policy_loss + entropy_loss
            loss.backward()
            nn.utils.clip_grad_norm_(net.policy.parameters(), CLIP)
            
            opt.step()
            
            batch_cnt = 0
            batch_obs = []
            batch_act = []
            batch_qval = []
    env.close()

C:\Users\BK\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


epoch : 0 count : 1000 rew : -1
epoch : 1 count : 1000 rew : -1
epoch : 2 count : 1000 rew : -1
epoch : 3 count : 1000 rew : -1
epoch : 4 count : 1000 rew : -1
epoch : 5 count : 1000 rew : -1
epoch : 6 count : 1000 rew : -1
epoch : 7 count : 1000 rew : -1
epoch : 8 count : 1000 rew : -1
epoch : 9 count : 1000 rew : -1
epoch : 10 count : 1000 rew : -1
epoch : 11 count : 1000 rew : -1
epoch : 12 count : 1000 rew : -1
epoch : 13 count : 1000 rew : -1
epoch : 14 count : 1000 rew : -1
epoch : 15 count : 1000 rew : -1
epoch : 16 count : 1000 rew : -1
epoch : 17 count : 1000 rew : -1
epoch : 18 count : 1000 rew : -1
epoch : 19 count : 1000 rew : -1
epoch : 20 count : 1000 rew : -1
epoch : 21 count : 1000 rew : -1
epoch : 22 count : 1000 rew : -1
epoch : 23 count : 1000 rew : -1
epoch : 24 count : 1000 rew : -1
epoch : 25 count : 1000 rew : -1
epoch : 26 count : 1000 rew : -1
epoch : 27 count : 1000 rew : -1
epoch : 28 count : 1000 rew : -1
epoch : 29 count : 1000 rew : -1
epoch : 30 count : 1

epoch : 244 count : 1000 rew : -1
epoch : 245 count : 1000 rew : -1
epoch : 246 count : 1000 rew : -1
epoch : 247 count : 1000 rew : -1
epoch : 248 count : 1000 rew : -1
epoch : 249 count : 1000 rew : -1
epoch : 250 count : 1000 rew : -1
epoch : 251 count : 1000 rew : -1
epoch : 252 count : 1000 rew : -1
epoch : 253 count : 1000 rew : -1
epoch : 254 count : 1000 rew : -1
epoch : 255 count : 1000 rew : -1
epoch : 256 count : 1000 rew : -1
epoch : 257 count : 1000 rew : -1
epoch : 258 count : 1000 rew : -1
epoch : 259 count : 1000 rew : -1
epoch : 260 count : 1000 rew : -1
epoch : 261 count : 1000 rew : -1
epoch : 262 count : 1000 rew : -1
epoch : 263 count : 1000 rew : -1
epoch : 264 count : 1000 rew : -1
epoch : 265 count : 1000 rew : -1
epoch : 266 count : 1000 rew : -1
epoch : 267 count : 1000 rew : -1
epoch : 268 count : 1000 rew : -1
epoch : 269 count : 1000 rew : -1
epoch : 270 count : 1000 rew : -1
epoch : 271 count : 1000 rew : -1
epoch : 272 count : 1000 rew : -1
epoch : 273 co

epoch : 484 count : 1000 rew : -1
epoch : 485 count : 1000 rew : -1
epoch : 486 count : 1000 rew : -1
epoch : 487 count : 1000 rew : -1
epoch : 488 count : 1000 rew : -1
epoch : 489 count : 1000 rew : -1
epoch : 490 count : 1000 rew : -1
epoch : 491 count : 1000 rew : -1
epoch : 492 count : 1000 rew : -1
epoch : 493 count : 1000 rew : -1
epoch : 494 count : 1000 rew : -1
epoch : 495 count : 1000 rew : -1
epoch : 496 count : 1000 rew : -1
epoch : 497 count : 1000 rew : -1
epoch : 498 count : 1000 rew : -1
epoch : 499 count : 1000 rew : -1
epoch : 500 count : 1000 rew : -1
epoch : 501 count : 1000 rew : -1
epoch : 502 count : 1000 rew : -1
epoch : 503 count : 1000 rew : -1
epoch : 504 count : 1000 rew : -1
epoch : 505 count : 1000 rew : -1
epoch : 506 count : 1000 rew : -1
epoch : 507 count : 1000 rew : -1
epoch : 508 count : 1000 rew : -1
epoch : 509 count : 1000 rew : -1
epoch : 510 count : 1000 rew : -1
epoch : 511 count : 1000 rew : -1
epoch : 512 count : 1000 rew : -1
epoch : 513 co

epoch : 725 count : 1000 rew : -1
epoch : 726 count : 1000 rew : -1
epoch : 727 count : 1000 rew : -1
epoch : 728 count : 1000 rew : -1
epoch : 729 count : 1000 rew : -1
epoch : 730 count : 1000 rew : -1
epoch : 731 count : 1000 rew : -1
epoch : 732 count : 1000 rew : -1
epoch : 733 count : 1000 rew : -1
epoch : 734 count : 1000 rew : -1
epoch : 735 count : 1000 rew : -1
epoch : 736 count : 1000 rew : -1
epoch : 737 count : 1000 rew : -1
epoch : 738 count : 1000 rew : -1
epoch : 739 count : 1000 rew : -1
epoch : 740 count : 1000 rew : -1
epoch : 741 count : 1000 rew : -1
epoch : 742 count : 1000 rew : -1
epoch : 743 count : 1000 rew : -1
epoch : 744 count : 1000 rew : -1
epoch : 745 count : 1000 rew : -1
epoch : 746 count : 1000 rew : -1
epoch : 747 count : 1000 rew : -1
epoch : 748 count : 1000 rew : -1
epoch : 749 count : 1000 rew : -1
epoch : 750 count : 1000 rew : -1
epoch : 751 count : 1000 rew : -1
epoch : 752 count : 1000 rew : -1
epoch : 753 count : 1000 rew : -1
epoch : 754 co

...

epoch : 1200 count : 1000 rew : -1
epoch : 1201 count : 1000 rew : -1
epoch : 1202 count : 1000 rew : -1
epoch : 1203 count : 1000 rew : -1
epoch : 1204 count : 1000 rew : -1
epoch : 1205 count : 1000 rew : -1
epoch : 1206 count : 1000 rew : -1
epoch : 1207 count : 1000 rew : -1
epoch : 1208 count : 1000 rew : -1
epoch : 1209 count : 1000 rew : -1
epoch : 1210 count : 1000 rew : -1
epoch : 1211 count : 1000 rew : -1
epoch : 1212 count : 1000 rew : -1
epoch : 1213 count : 1000 rew : -1
epoch : 1214 count : 1000 rew : -1
epoch : 1215 count : 1000 rew : -1
epoch : 1216 count : 1000 rew : -1
epoch : 1217 count : 1000 rew : -1
epoch : 1218 count : 1000 rew : -1
epoch : 1219 count : 1000 rew : -1
epoch : 1220 count : 1000 rew : -1
epoch : 1221 count : 1000 rew : -1
epoch : 1222 count : 1000 rew : -1
epoch : 1223 count : 1000 rew : -1
epoch : 1224 count : 1000 rew : -1
epoch : 1225 count : 1000 rew : -1
epoch : 1226 count : 1000 rew : -1
epoch : 1227 count : 1000 rew : -1
epoch : 1228 count :

epoch : 1434 count : 1000 rew : -1
epoch : 1435 count : 1000 rew : -1
epoch : 1436 count : 1000 rew : -1
epoch : 1437 count : 1000 rew : -1
epoch : 1438 count : 1000 rew : -1
epoch : 1439 count : 1000 rew : -1
epoch : 1440 count : 1000 rew : -1
epoch : 1441 count : 1000 rew : -1
epoch : 1442 count : 1000 rew : -1
epoch : 1443 count : 1000 rew : -1
epoch : 1444 count : 1000 rew : -1
epoch : 1445 count : 1000 rew : -1
epoch : 1446 count : 1000 rew : -1
epoch : 1447 count : 1000 rew : -1
epoch : 1448 count : 1000 rew : -1
epoch : 1449 count : 1000 rew : -1
epoch : 1450 count : 1000 rew : -1
epoch : 1451 count : 1000 rew : -1
epoch : 1452 count : 1000 rew : -1
epoch : 1453 count : 1000 rew : -1
epoch : 1454 count : 1000 rew : -1
epoch : 1455 count : 1000 rew : -1
epoch : 1456 count : 1000 rew : -1
epoch : 1457 count : 1000 rew : -1
epoch : 1458 count : 1000 rew : -1
epoch : 1459 count : 1000 rew : -1
epoch : 1460 count : 1000 rew : -1
epoch : 1461 count : 1000 rew : -1
epoch : 1462 count :

epoch : 1668 count : 1000 rew : -1
epoch : 1669 count : 1000 rew : -1
epoch : 1670 count : 1000 rew : -1
epoch : 1671 count : 1000 rew : -1
epoch : 1672 count : 1000 rew : -1
epoch : 1673 count : 1000 rew : -1
epoch : 1674 count : 1000 rew : -1
epoch : 1675 count : 1000 rew : -1
epoch : 1676 count : 1000 rew : -1
epoch : 1677 count : 1000 rew : -1
epoch : 1678 count : 1000 rew : -1
epoch : 1679 count : 1000 rew : -1
epoch : 1680 count : 1000 rew : -1
epoch : 1681 count : 1000 rew : -1
epoch : 1682 count : 1000 rew : -1
epoch : 1683 count : 1000 rew : -1
epoch : 1684 count : 1000 rew : -1
epoch : 1685 count : 1000 rew : -1
epoch : 1686 count : 1000 rew : -1
epoch : 1687 count : 1000 rew : -1
epoch : 1688 count : 1000 rew : -1
epoch : 1689 count : 1000 rew : -1
epoch : 1690 count : 1000 rew : -1
epoch : 1691 count : 1000 rew : -1
epoch : 1692 count : 1000 rew : -1
epoch : 1693 count : 1000 rew : -1
epoch : 1694 count : 1000 rew : -1
epoch : 1695 count : 1000 rew : -1
epoch : 1696 count :

epoch : 1902 count : 1000 rew : -1
epoch : 1903 count : 1000 rew : -1
epoch : 1904 count : 1000 rew : -1
epoch : 1905 count : 1000 rew : -1
epoch : 1906 count : 1000 rew : -1
epoch : 1907 count : 1000 rew : -1
epoch : 1908 count : 1000 rew : -1
epoch : 1909 count : 1000 rew : -1
epoch : 1910 count : 1000 rew : -1
epoch : 1911 count : 1000 rew : -1
epoch : 1912 count : 1000 rew : -1
epoch : 1913 count : 1000 rew : -1
epoch : 1914 count : 1000 rew : -1
epoch : 1915 count : 1000 rew : -1
epoch : 1916 count : 1000 rew : -1
epoch : 1917 count : 1000 rew : -1
epoch : 1918 count : 1000 rew : -1
epoch : 1919 count : 1000 rew : -1
epoch : 1920 count : 1000 rew : -1
epoch : 1921 count : 1000 rew : -1
epoch : 1922 count : 1000 rew : -1
epoch : 1923 count : 1000 rew : -1
epoch : 1924 count : 1000 rew : -1
epoch : 1925 count : 1000 rew : -1
epoch : 1926 count : 1000 rew : -1
epoch : 1927 count : 1000 rew : -1
epoch : 1928 count : 1000 rew : -1
epoch : 1929 count : 1000 rew : -1
epoch : 1930 count :

epoch : 2136 count : 1000 rew : -1
epoch : 2137 count : 1000 rew : -1
epoch : 2138 count : 1000 rew : -1
epoch : 2139 count : 1000 rew : -1
epoch : 2140 count : 1000 rew : -1
epoch : 2141 count : 1000 rew : -1
epoch : 2142 count : 1000 rew : -1
epoch : 2143 count : 1000 rew : -1
epoch : 2144 count : 1000 rew : -1
epoch : 2145 count : 1000 rew : -1
epoch : 2146 count : 1000 rew : -1
epoch : 2147 count : 1000 rew : -1
epoch : 2148 count : 1000 rew : -1
epoch : 2149 count : 1000 rew : -1
epoch : 2150 count : 1000 rew : -1
epoch : 2151 count : 1000 rew : -1
epoch : 2152 count : 1000 rew : -1
epoch : 2153 count : 1000 rew : -1
epoch : 2154 count : 1000 rew : -1
epoch : 2155 count : 1000 rew : -1
epoch : 2156 count : 1000 rew : -1
epoch : 2157 count : 1000 rew : -1
epoch : 2158 count : 1000 rew : -1
epoch : 2159 count : 1000 rew : -1
epoch : 2160 count : 1000 rew : -1
epoch : 2161 count : 1000 rew : -1
epoch : 2162 count : 1000 rew : -1
epoch : 2163 count : 1000 rew : -1
epoch : 2164 count :

In [ ]:
frame = []
obs = env.reset()
frame.append(env.render(mode="rgb_array"))

while True:
    with torch.no_grad():
        out, _ = net(torch.FloatTensor(obs))
        act = np.random.choice(env.action_space.n, p=F.softmax(out).numpy())

    next_obs, _, done, _ = env.step(act)
    frame.append(env.render(mode="rgb_array"))
    obs = next_obs
    
    if done:
        break
env.close()
len(frame)

In [ ]:
#frame_player.display_frames_as_gif(frame, 60)

In [ ]:
EPOCH = 5000